In [1]:
exec(open("init_notebook.py", "r").read())

current working dirC:\Users\abjaw\Documents\GitHub\multi-region-attention


In [2]:
from models.encoders.mra_helper import OverlapPatchEmbed, Mlp, DWConv, Block
from models.encoders.mra_transformer import MRATransformer
from models.decoders.MLPDecoder import DecoderHead
from models.encoders.merge_attn import MultiScaleAttention
from functools import partial
import torch.nn as nn
import torch

In [27]:
# overlap_patch_embed = OverlapPatchEmbed(patch_size=7, 
#                                         stride=4, 
#                                         in_chans=3, 
#                                         embed_dim=64)

# mlp = Mlp(in_features=64,
#           hidden_features=64*4,
#           act_layer=torch.nn.GELU,
#           drop=0)

# multiscale_attn = MultiScaleAttention(dim=64,
#                                       num_heads=2,
#                                       sr_ratio=8,
#                                       local_region_shape=[8, 16],
#                                       img_size=(256, 256)) 

# block = Block(dim=64,
#               num_heads=2, 
#               mlp_ratio=4,
#               qkv_bias=True,
#               qk_scale=None,
#               drop=0,
#               attn_drop=0, 
#               drop_path=0,
#               norm_layer=torch.nn.LayerNorm,
#               sr_ratio=8,
#               local_region_shape=[8, 16],
#               img_size=(256, 256))


mra_transformer = MRATransformer(img_size=(512, 512), patch_size=4, 
                                   embed_dims=[64, 128, 320, 512], 
                                   num_heads=[2, 4, 5, 8], 
                                   mlp_ratios=[4, 4, 4, 4], 
                                   qkv_bias=True, 
                                   norm_layer=partial(nn.LayerNorm, eps=1e-6), 
                                   depths=[3,4,6,3], 
                                   sr_ratios=[8, 4, 2, 1], 
                                   drop_rate=0.0, drop_path_rate=0.1)
# in_channels=self.channels, num_classes=cfg.num_classes, norm_layer=norm_layer, embed_dim=cfg.decoder_embed_dim
mlp_decoder = DecoderHead(in_channels =  [64, 128, 320, 512],
                         num_classes = 19,
                         norm_layer = nn.BatchNorm2d,
                         embed_dim = 512)

In [28]:


x_6 = torch.rand(1, 3, 512, 512)
y_6 = mra_transformer(x_6)

# torch.Size([1, 64, 256, 256]) torch.Size([1, 128, 128, 128]) torch.Size([1, 320, 64, 64]) torch.Size([1, 512, 32, 32])
x_7 = [torch.rand(1, 64, 128, 128), torch.rand(1, 128, 64, 64), torch.rand(1, 320, 32, 32), torch.rand(1, 512, 16, 16)]
y_7 = mlp_decoder(x_7)


mra_flops = mra_transformer.flops()
mlp_flops = mlp_decoder.flops()
# print(rgbx_transformer.flops())
# print(mlp_decoder.flops())

print("encoder ", mra_flops)
print("mlp ", mlp_flops)

print("total ", mra_flops + mlp_flops)

number of heads  2
number of heads  2
number of heads  2
number of heads  4
number of heads  4
number of heads  4
number of heads  4
number of heads  5
number of heads  5
number of heads  5
number of heads  5
number of heads  5
number of heads  5
number of heads  8
number of heads  8
number of heads  8
encoder  5371224064.0
mlp  34687942656
total  40059166720.0
